In [1]:
# for download and decompress the dataset
import os, zipfile, glob, io, requests

# control
import numpy as np

import pandas as pd
import geopandas as gpd
from osgeo import gdal
import georasters as gr

#plotting
import matplotlib.pyplot as plt


# Useful Functions

For fetching raster from Hansen

In [7]:
# Fetch hansen datasets
def fetch_raster(list_url, country_name):
    ''' Fetcht the rasters by from a list url, download it and 
    subset 2 rasters:
    - url : raster dataset url from Hansen
    - country_name: to keep separated folders by country name'''

    # main_folder
    current_folder = os.getcwd()
    new_folder = str(country_name)
    
    # arrangements
    path = os.path.join(current_folder, new_folder)
    if os.path.isdir(path) == False:
        os.mkdir(path) # create folder
        
    os.chdir(path) # move to folder

    # Fetching geotif form Hansen
    for url in list_url:
        fetching_url = f'! wget {url} -q'
        if os.path.exists(url.rsplit('/')[-1]) == False:
            os.system(fetching_url)

    # print the outcome
    print('%2d raster files downloaded in folder %s.' %(len(list_url), new_folder))

    # return to working directory
    os.chdir(current_folder)

Clipping shapefiles (to clip rasters)

In [16]:
# importing the data
world_df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# subseting columns
world_df = world_df[['continent','name', 'iso_a3','geometry']]

# croppping boundaries and save shapefiles
path = os.path.join(os.getcwd(), 'shapefiles')
if os.path.isdir(path) == False:
    os.mkdir(path) # create folder

for i in ['Colombia', 'Belize']:
    temp = world_df.loc[world_df.name == i, ].reset_index(drop=True)
    temp.to_file("./shapefiles/" + i.lower() + ".shp")

# load the shapefiles from cropping
colombia_shp = './shapefiles/colombia.shp'
belize_shp = './shapefiles/belize.shp'

Clip raster using shapefile as borderline

In [39]:
# Function to clip and save rasters
def clip_raster(merged_raster, inshp, final_name):
    ''' Get a clipped raster from the Worldclim data for specified countries:
    - merged_raster : geotiff merged under the country borderline
    - inshp : path clipped shapefile with the name of the country
    - final_name: final name of the raster/folder for clipped outraster '''
    path = os.path.join(os.getcwd(), final_name)
    if os.path.isdir(path) == False:
        os.mkdir(path) # create folder

    outraster = './' + final_name + '/' + str(final_name) + '.tif'
    dsClip = gdal.Warp(outraster, raster, cutlineDSName=inshp, cropToCutline=True, dstNodata=np.nan)
  
    # print the results
    print('Rasters succesfully clipped and stored at the folder %s.' %(final_name))
    dsClip= None

# Fetching rasters

In [41]:
# URLs
colombia_treecover = [
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_10N_080W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_20N_080W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_10N_070W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_00N_080W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_00N_070W.tif',
    'https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_20N_070W.tif'
    ]

belize_treecover = ['https://storage.googleapis.com/earthenginepartners-hansen/GFC-2019-v1.7/Hansen_GFC-2019-v1.7_treecover2000_20N_090W.tif']

# Fetching datasets
fetch_raster(colombia_treecover, 'colombia')
fetch_raster(belize_treecover, 'belize')


 6 raster files downloaded in folder colombia.
 1 raster files downloaded in folder belize.


# Tree cover in Colombia

In [43]:
# Getting path
path = os.path.join(os.getcwd(), 'colombia')

# Read the data
files = glob.glob(os.path.join(path, '*tif')) # list of files in folder
names = [i[i.rfind('treecover2000'):].rsplit('.')[0] for i in files] # name for read data
colombia = list(zip(names, [gdal.Open(f) for f in files]))

In [44]:
# Review
for i in colombia:
    name, raster = i
    print('File: ', name)

    # Projection and Raster Metadata
    # print(raster.GetProjection(), raster.GetMetadata())

    # Dimensions X, Y and Number of bands
    print(raster.RasterXSize, raster.RasterYSize, raster.RasterCount)
    print()
    

File:  treecover2000_10N_080W
40000 40000 1

File:  treecover2000_20N_080W
40000 40000 1

File:  treecover2000_00N_070W
40000 40000 1

File:  treecover2000_20N_070W
40000 40000 1

File:  treecover2000_00N_080W
40000 40000 1

File:  treecover2000_10N_070W
40000 40000 1



In [45]:
# Merging the rasters
inputs = ' '.join(files)
output = os.path.join(path, 'colombia-merged__SouthAmerica.tif')

# Command
command = "gdal_merge.py %s -o %s" % (inputs, output)

# Run the command
os.system(command)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [48]:
# Clipping the rasters
merged = gdal.Open(output)
# clip_raster(merged, colombia_shp, 'colombia_treecover2000')

In [52]:
# Projection and Raster Metadata
# print(merged.GetProjection(), merged.GetMetadata())

# Dimensions X, Y and Number of bands
print(merged.RasterXSize, merged.RasterYSize, merged.RasterCount)
print()

80000 120000 1

